In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 6

In [2]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"thnhnguyenphuc","key":"c4567d6a802976fa686fb1f293c5ce5f"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
from kaggle.api.kaggle_api_extended import KaggleApi

# https://technowhisp.com/kaggle-api-python-documentation/
# Authenticate to Kaggle
api = KaggleApi()
api.authenticate()

# Download the dataset
api.dataset_download_files('duyminhnguyentran/csc15105', unzip=True)

In [4]:
file_path_train = "Project2_Data/train.json"
file_path_test = "Project2_Data/test.json"

In [5]:
import json

with open(file_path_train, "r") as read_file:
    train_data = json.load(read_file)

with open(file_path_test, "r") as read_file:
    test_data = json.load(read_file)

In [6]:
for item in train_data:
    for qas in item['qas']:
        if qas['answers'][0]['answer_start'] == -1:
            qas['answers'][0]['answer_start'] = item['context'].find(qas['answers'][0]['text'])
    item['qas'] = [qas for qas in item['qas'] if len(qas['answers'][0]['text']) >= 1 or qas['answers'][0]['text'].isdigit()]
    item['qas'] = [qas for qas in item['qas'] if len(qas['question']) >= 1]
with open('modified_train_data.json', 'w') as file:
    json.dump(train_data, file, indent=4)

for item in test_data:
    for qas in item['qas']:
        if qas['answers'][0]['answer_start'] == -1:
            qas['answers'][0]['answer_start'] = item['context'].find(qas['answers'][0]['text'])
    item['qas'] = [qas for qas in item['qas'] if len(qas['answers'][0]['text']) >= 1 or qas['answers'][0]['text'].isdigit()]
    item['qas'] = [qas for qas in item['qas'] if len(qas['question']) >= 1]
with open('modified_test_data.json', 'w') as file:
    json.dump(test_data, file, indent=4)

In [7]:
file_path_train = "modified_train_data.json"
file_path_test = "modified_test_data.json"

In [8]:
import json

with open(file_path_train, "r") as read_file:
    train = json.load(read_file)

with open(file_path_test, "r") as read_file:
    test = json.load(read_file)

In [9]:
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [10]:
model_type= "squeezebert"
model_name= "squeezebert/squeezebert-uncased"


In [11]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 32
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5

In [12]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    "train_batch_size": 128,
    "eval_batch_size": 32,
}


In [13]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/500 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/103M [00:00<?, ?B/s]

Some weights of SqueezeBertForQuestionAnswering were not initialized from the model checkpoint at squeezebert/squeezebert-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [14]:
model.train_model(train, eval_data=test)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
add example index and unique id: 100%|██████████| 49322/49322 [00:00<00:00, 811722.20it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


...

Running Evaluation:   0%|          | 0/517 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/404 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/517 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/517 [00:00<?, ?it/s]

(2020,
 {'global_step': [404, 808, 1000, 1212, 1616, 2000, 2020],
  'correct': [257, 466, 472, 669, 883, 921, 907],
  'similar': [13945, 12929, 13305, 11984, 11061, 11077, 11145],
  'incorrect': [1568, 2375, 1993, 3117, 3826, 3772, 3718],
  'train_loss': [3.152134418487549,
   2.6153225898742676,
   2.645535469055176,
   2.6166505813598633,
   2.81076979637146,
   2.3024234771728516,
   2.985743284225464],
  'eval_loss': [-1.5001700012088974,
   -1.45131165377176,
   -1.8281136665860735,
   -1.593385441852031,
   -1.6244201069874276,
   -1.7510804521276595,
   -1.7658346681576402]})

In [15]:
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/517 [00:00<?, ?it/s]

In [16]:
result

{'correct': 907,
 'similar': 11145,
 'incorrect': 3718,
 'eval_loss': -1.7658346681576402}

In [17]:
texts

{'correct_text': {'5733b2fe4776f41900661090': '1928',
  '5733b5df4776f41900661109': '12.179',
  '5733b5df4776f41900661106': '21-24%',
  '5733b699d058e614000b611a': '15',
  '573394c84776f41900660ddf': '1240',
  '573398ebd058e614000b5e67': '1950',
  '57339a5bd058e614000b5e91': '29',
  '5733849bd058e614000b5c59': 'ngày 26 tháng 11 năm 1842,',
  '573385394776f41900660c7f': '1849',
  '5733c0064776f4190066119a': 'WSND-FM',
  '56cef8faaab44d1400b88d6a': '04 tháng 9 năm 1981)',
  '56d43da72ccc5a1400d830be': "B'Day",
  '56d43da72ccc5a1400d830bf': 'Dreamgirls',
  '56bf7cb63aeaaa14008c9677': 'năm 1996,',
  '56bf91c6a10cfb140055117f': 'tháng 11 năm 2003,',
  '56d4bf242ccc5a1400d831be': 'ngày 04 tháng tư năm 2008,',
  '56bf9c70a10cfb14005511bb': 'năm 2010,',
  '56d4c4532ccc5a1400d83205': 'ngày 28 tháng 6 năm 2011',
  '56be9bb83aeaaa14008c915c': 'Tháng Một 7, 2012,',
  '56bfa087a10cfb14005511d9': 'Tháng Một 7, 2012,',
  '56d4c4e72ccc5a1400d83218': 'Tháng Một 7, 2012,',
  '56bfa24ea10cfb14005511e3': 

In [18]:
# Make predictions with the model
to_predict = [
    {
        "context": "Bồ Đào Nha có nguồn tài nguyên đáng kể gió và điện sông, hai nguồn năng lượng tái tạo hiệu quả nhất. Kể từ thời điểm chuyển giao thế kỷ 21, đã có một xu hướng phát triển của một ngành công nghiệp tài nguyên tái tạo và giảm cả tiêu thụ và sử dụng tài nguyên nhiên liệu hóa thạch. Năm 2006, nhà máy điện năng lượng mặt trời lớn nhất thế giới tại thời điểm đó, nhà máy điện Moura quang điện, bắt đầu hoạt động gần Moura, ở phía nam, trong khi trang trại năng lượng sóng thương mại đầu tiên trên thế giới, Aguçadoura sóng trại , mở trong khu vực Norte  (2008). Đến cuối năm 2006, 66% sản lượng điện của nước này là từ nhà máy than và điện nhiên liệu, trong khi 29% được lấy từ đập thủy điện, và 6% bằng năng lượng gió. .",
        "qas": [
            {
                "question": "hai nguồn đáng kể nhất của năng lượng tái tạo ở Bồ Đào Nha là gì?",
                "id": "0",
            }
        ],
    }
]

In [19]:
answers, probabilities = model.predict(to_predict)
print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3581.81it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['Bồ Đào Nha có nguồn tài nguyên đáng kể gió và điện sông, hai nguồn năng lượng tái tạo hiệu quả nhất.', 'Bồ Đào Nha có nguồn tài nguyên đáng kể gió và điện sông,', 'quả nhất.']}]
